In [1]:
#setup - rememeber to switch to tensorflow 2.3 kernel...
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(tf.__version__)

%matplotlib notebook
%load_ext tensorboard
%load_ext autoreload
%autoreload 2
%autosave 180

C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


2.3.0


Autosaving every 180 seconds


In [2]:
#load individual data numpy files
#-------------------------------------------------------------------
# d1_1 = np.load('C:/Users/Derm/Desktop/big/pshift/test1_scan1.npy')
# d2_1 = np.load('C:/Users/Derm/Desktop/big/pshift/test1_scan2.npy')
# y1 = np.load('C:/Users/Derm/Desktop/big/pshift/test1_ground_truth.npy')

# d1_2 = np.load('C:/Users/Derm/Desktop/big/pshift/test2_scan1.npy')
# d2_2 = np.load('C:/Users/Derm/Desktop/big/pshift/test2_scan2.npy')
# y2 = np.load('C:/Users/Derm/Desktop/big/pshift/test2_ground_truth.npy')

d1_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_scan1.npy")
d2_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_scan2.npy")
gt_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_ground_truth.npy")

d1_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan1.txt")
d2_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan2.txt")
gt_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_ground_truth.txt")

d1 = np.append(d1_1, d1_2, axis = 0)
d2 = np.append(d2_1, d2_2, axis = 0)
gt = np.append(gt_1, gt_2, axis = 0)

# #TODO: loop through all files in testing folder 
# #combine scans 1 and 2 from both files
#-------------------------------------------------------------------
#single big file
# d1 = np.load("C:/Users/Derm/Desktop/big/pshift/scan1_300k_50_samples.npy")
# d2 = np.load("C:/Users/Derm/Desktop/big/pshift/scan2_300k_50_samples.npy")
# gt = np.load("C:/Users/Derm/Desktop/big/pshift/ground_truth_300k_50_samples.npy")
#-------------------------------------------------------------------

#reshape but don't convert to tensor
points_per_sample = 50          #poitns sammpled from each voxel
tsplit = 0.95                   #this fraction goes into training

scan1 = np.reshape(d1, [-1, points_per_sample, 3])
scan2 = np.reshape(d2, [-1, points_per_sample, 3])
ntrain = int(tsplit*tf.shape(scan1)[0].numpy())

x_train = np.append(scan1[:ntrain], scan2[:ntrain], axis = 1)
x_test = np.append(scan1[ntrain:], scan2[ntrain:], axis = 1)
# print(np.shape(x_train))
# print(np.shape(x_test))

y_train = gt[:ntrain]
y_test = gt[ntrain:]
# print(np.shape(y_train))
# print(np.shape(y_test))

#-------------------------------------------------------------------



In [3]:
#train network
from network import Net

runLen = 300

def scheduler(epoch, learning_rate):
    part1 = runLen//3
    part2 = 2*runLen//3
    if epoch < part1:
        learning_rate = 0.002
        return learning_rate
    if epoch >= part1 and epoch < part2:
        learning_rate = 0.001
        return learning_rate
    if epoch >= part2:
        learning_rate = 0.00025
        return learning_rate

model = Net()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005),
              loss = tf.keras.losses.MeanAbsoluteError()) #was MeanSquaredError()

summary = model.summary()
print(summary)

scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
cp = tf.keras.callbacks.ModelCheckpoint("FordNetCP.kmod", monitor = 'val_loss', save_best_only = True) 

log_dir = "runs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

trace = model.fit(x = x_train, y = y_train, batch_size = 512, epochs=runLen, verbose=1, 
                  validation_split = 0.2, shuffle=True, callbacks = [cp, scheduler])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 3)]          0         
_________________________________________________________________
tf_op_layer_ExpandDims (Tens [(None, 100, 3, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 1, 64)        256       
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 1, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 1, 64)        4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 1, 64)        256       
_________________________________________________________________
tf_op_layer_Reshape (TensorF [(None, 100, 64)]        

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: FordNetCP.kmod\assets
595/595 [==============================] - 10s 18ms/step - loss: 0.1518 - val_loss: 0.1492
Epoch 4/300
595/595 [==============================] - 9s 15ms/step - loss: 0.1435 - val_loss: 0.1626
Epoch 5/300
595/595 [==============================] - 9s 15ms/step - loss: 0.1354 - val_loss: 0.1498
Epoch 6/300
591/595 [============================>.] - ETA: 0s - loss: 0.1291WARNING:tensorflow:AutoGraph could not transform <function canonicalize_signatures.<locals>.signature_wrapper at 0x000001CA5ECCBD38> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux,

INFO:tensorflow:Assets written to: FordNetCP.kmod\assets
595/595 [==============================] - 10s 17ms/step - loss: 0.0904 - val_loss: 0.0703
Epoch 19/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0893 - val_loss: 0.0846
Epoch 20/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0878 - val_loss: 0.1244
Epoch 21/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0898 - val_loss: 0.0725
Epoch 22/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0862 - val_loss: 0.0865
Epoch 23/300
593/595 [============================>.] - ETA: 0s - loss: 0.0839WARNING:tensorflow:AutoGraph could not transform <function canonicalize_signatures.<locals>.signature_wrapper at 0x000001CA4E075948> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'Non

595/595 [==============================] - 9s 15ms/step - loss: 0.0737 - val_loss: 0.0730
Epoch 46/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0725 - val_loss: 0.0750
Epoch 47/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0719 - val_loss: 0.0612
Epoch 48/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0718 - val_loss: 0.0723
Epoch 49/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0715 - val_loss: 0.0535
Epoch 50/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0724 - val_loss: 0.0510
Epoch 51/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0707 - val_loss: 0.0568
Epoch 52/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0709 - val_loss: 0.0525
Epoch 53/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0703 - val_loss: 0.0521
Epoch 54/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0706 - 

595/595 [==============================] - 10s 17ms/step - loss: 0.0643 - val_loss: 0.0425
Epoch 95/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0640 - val_loss: 0.0518
Epoch 96/300
594/595 [============================>.] - ETA: 0s - loss: 0.0648WARNING:tensorflow:AutoGraph could not transform <function canonicalize_signatures.<locals>.signature_wrapper at 0x000001C9B6FD73A8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorat

593/595 [============================>.] - ETA: 0s - loss: 0.0582WARNING:tensorflow:AutoGraph could not transform <function canonicalize_signatures.<locals>.signature_wrapper at 0x000001C9B7E34678> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: FordNetCP.kmod\assets
595/595 [==============================] - 10s 17ms/step - loss: 0.0582 - val_loss: 0.03

595/595 [==============================] - 9s 15ms/step - loss: 0.0569 - val_loss: 0.0467
Epoch 191/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0565 - val_loss: 0.0395
Epoch 192/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0564 - val_loss: 0.0407
Epoch 193/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0572 - val_loss: 0.0511
Epoch 194/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0571 - val_loss: 0.0468
Epoch 195/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0566 - val_loss: 0.0370
Epoch 196/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0565 - val_loss: 0.0421
Epoch 197/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0564 - val_loss: 0.0480
Epoch 198/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0573 - val_loss: 0.0562
Epoch 199/300
595/595 [==============================] - 9s 15ms/step - loss: 

595/595 [==============================] - 9s 15ms/step - loss: 0.0535 - val_loss: 0.0323
Epoch 239/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0533 - val_loss: 0.0349
Epoch 240/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0531 - val_loss: 0.0355
Epoch 241/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0532 - val_loss: 0.0312
Epoch 242/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0533 - val_loss: 0.0304
Epoch 243/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0540 - val_loss: 0.0379
Epoch 244/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0533 - val_loss: 0.0303
Epoch 245/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0534 - val_loss: 0.0320
Epoch 246/300
595/595 [==============================] - 9s 15ms/step - loss: 0.0536 - val_loss: 0.0306
Epoch 247/300
595/595 [==============================] - 9s 15ms/step - loss: 